In [119]:
import numpy as np 
import pandas as pd
import logging 
from dateutil import parser
from time import time
import os
from dateutil import parser
import datetime
from datetime import timedelta

data_dir = "X:\\Data_Analytics\\MWPL\\data\\"

In [120]:
result = pd.read_excel('MasterData.xlsx')[['SYMBOL']]

for r,d,f in os.walk(data_dir): 
    for excel_file in f:  
        name= excel_file.split('_')[-1].split('.')[0]  # column name
        
        name = datetime.datetime.strptime(name, '%d%m%Y').date()
        df = pd.read_excel(data_dir+excel_file, skiprows=1).fillna(0)
        df.drop(columns=['Sr No.'], inplace=True)
        df.set_index('Underlying Stock', inplace=True)
        df = df.apply(pd.to_numeric)
        df['Sum'] = df.sum(axis=1, skipna=True)
        df = df.reindex(columns = ['Sum'])
        df.reset_index(inplace=True)
        df.rename(columns={'Underlying Stock':'SYMBOL'}, inplace=True)
        # merge with master and name the columns 
        result = result.merge(df, on='SYMBOL', how='left')        
        result.fillna(0, inplace=True)
        result.rename(columns={'Sum':'{}'.format(name)}, inplace=True)
        
# after processing all the files sort the file index and columns 
result.set_index('SYMBOL', inplace=True)
d = list(sorted([ datetime.datetime.strptime(x, "%Y-%m-%d").date() for x in result.columns.values ])) # get all the columns in sorted manner
result = result.reindex(columns = [ x.strftime("%Y-%m-%d") for x in d ]) # reindex columns to new sorted format 
result.reset_index(inplace=True)
result.sort_values(by='SYMBOL', inplace=True) # sort on symbols 
result.to_excel('MWPL_Report.xlsx', index=False)